In [1]:
import os
from langchain.text_splitter import CharacterTextSplitter , RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import chroma
from langchain_openai import OpenAIEmbeddings

In [6]:
# current_dir = os.path.dirname(os.path.abspath(__file__))
current_dir = os.getcwd()
file_path = os.path.join(current_dir,"text","test.txt")
persist_dir = os.path.join(current_dir,"db","chroma_db")

In [29]:
if not os.path.exists(file_path):
    raise FileNotFoundError(f"file {file_path} is not exists , please check the path")
loader = TextLoader(file_path)  
documents = loader.load()
text_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)
docs = text_split.split_documents(documents) 

In [30]:
print(f"number of documents : {len(docs)}")
print(f"sample chunk: \n{docs[0].page_content}\n")

number of documents : 66
sample chunk: 
Technology in Society 78 (2024) 102644
Available online 20 June 2024
0160-791X/© 2024 The Authors. Published by Elsevier Ltd. This is an open access article under the CC BY license ( http://creativecommons.org/licenses/by/4.0/ ).Addressing the notion of trust around ChatGPT in the high-stakes use case 
of insurance 
Juliane Ressela,*, Michaele Vollerb, Finbarr Murphya, Martin Mullinsa 
aKemmy Business School, University of Limerick, Limerick, V94 T9PX, Ireland

